<a href="https://colab.research.google.com/github/dpmaloney/miscprojects/blob/master/NameRBFTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [0]:
path = "NationalNames.csv"
names = pd.read_csv(path)
names.fillna(0)
names.head()

names.describe()



FileNotFoundError: ignored

In [0]:
namechart = names.groupby(['Name', 'Gender'], as_index = False)['Count'].sum()

namechartdiff = namechart.reset_index().pivot('Name', 'Gender', 'Count')
namechartdiff = namechartdiff.fillna(0)
namechartdiff["Mpercent"] = ((namechartdiff["M"] - namechartdiff["F"])/(namechartdiff["M"] + namechartdiff["F"]))
namechartdiff['gender'] = np.where(namechartdiff['Mpercent'] > 0.001, 'male', 'female')


char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
X = char_vectorizer.fit_transform(namechartdiff.index)
X = X.tocsc()
y = (namechartdiff.gender == 'male').values.astype(np.int)

itrain, itest = train_test_split(range(namechartdiff.shape[0]), train_size=0.7)
mask=np.ones(namechartdiff.shape[0], dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
X_train = X[mask]
Y_train =y[mask]
X_test =X[~mask]
Y_test =y[~mask]

In [0]:
def classifyRbf(gammaSetting):
  clf = SVC(kernel='rbf', gamma=gammaSetting)
  clf.fit(X_train, Y_train)
  return clf.score(X_test, Y_test)

In [0]:
for x in np.linspace(.1,1,10):
  print("RBF Kernel with gamma setting: ", x)
  print("Accuracy: ", classifyRbf(x))

RBF Kernel with gamma setting:  0.1
Accuracy:  0.8608905764584053
RBF Kernel with gamma setting:  0.2
Accuracy:  0.8775745023587619
RBF Kernel with gamma setting:  0.30000000000000004
Accuracy:  0.8786100563801634
RBF Kernel with gamma setting:  0.4
Accuracy:  0.8745829018524911
RBF Kernel with gamma setting:  0.5
Accuracy:  0.867334023702681
RBF Kernel with gamma setting:  0.6
Accuracy:  0.8560579910251985
RBF Kernel with gamma setting:  0.7000000000000001
Accuracy:  0.8406397422621102
RBF Kernel with gamma setting:  0.8
Accuracy:  0.8210792774134161
RBF Kernel with gamma setting:  0.9
Accuracy:  0.7986422736163847
RBF Kernel with gamma setting:  1.0
Accuracy:  0.7772408238407548
